In [1]:
import pandas as pd
import requests
import numpy as np
import time
import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import lxml
import os
import glob
from shapely.geometry import Point
from adjustText import adjust_text
from bs4 import BeautifulSoup
from tqdm import tqdm
from tqdm import trange
from datetime import datetime

%matplotlib inline

In [2]:
    ## CAMADA TRUSTED

In [21]:
import pandas as pd

df = gp.read_file('C:/Users/crash/Desktop/raw_speedtest_mobile.shp')

In [22]:
df.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,fonte,geometry
0,0323230233223102,15591,17249,26,4,2,138,1400100,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-60.75439 2.84978, -60.74890 2.84978..."
1,0323230233232230,16998,20375,25,2,1,138,1400100,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-60.72144 2.82234, -60.71594 2.82234..."
2,0323232011001023,17268,8038,29,2,2,138,1400100,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-60.77087 2.79491, -60.76538 2.79491..."
3,0323230233232011,116923,4563,36,1,1,138,1400100,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-60.71594 2.85526, -60.71045 2.85526..."
4,0323230233232033,5092,7704,31,2,2,138,1400100,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-60.71594 2.83880, -60.71045 2.83880..."


In [23]:
df['latitude'] = df.geometry.centroid.y
df['longitude'] = df.geometry.centroid.x
df.head()

C:\Users\crash\AppData\Local\Temp\ipykernel_18552\2334890284.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['latitude'] = df.geometry.centroid.y
C:\Users\crash\AppData\Local\Temp\ipykernel_18552\2334890284.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['longitude'] = df.geometry.centroid.x


,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,fonte,geometry,latitude,longitude
0,0323230233223102,15591,17249,26,4,2,138,1400100,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-60.75439 2.84978, -60.74890 2.84978...",2.847033,-60.751648
1,0323230233232230,16998,20375,25,2,1,138,1400100,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-60.72144 2.82234, -60.71594 2.82234...",2.819601,-60.718689
2,0323232011001023,17268,8038,29,2,2,138,1400100,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-60.77087 2.79491, -60.76538 2.79491...",2.792168,-60.768127
3,0323230233232011,116923,4563,36,1,1,138,1400100,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-60.71594 2.85526, -60.71045 2.85526...",2.852520,-60.713196
4,0323230233232033,5092,7704,31,2,2,138,1400100,https://ookla-open-data.s3-us-west-2.amazonaws...,"POLYGON ((-60.71594 2.83880, -60.71045 2.83880...",2.836060,-60.713196


In [6]:
# verificar se alguma coluna tem valores nulos
print(df.isnull().any())

quadkey       False
avg_d_kbps    False
avg_u_kbps    False
avg_lat_ms    False
tests         False
devices       False
index_righ    False
CD_MUN        False
fonte         False
geometry      False
latitude      False
longitude     False
dtype: bool


In [7]:
# contar o número de valores nulos em cada coluna
print(df.isnull().sum())

quadkey       0
avg_d_kbps    0
avg_u_kbps    0
avg_lat_ms    0
tests         0
devices       0
index_righ    0
CD_MUN        0
fonte         0
geometry      0
latitude      0
longitude     0
dtype: int64


In [8]:
# quantidade total de linhas
print(df.shape)

(588373, 12)


In [9]:
# todos os campos numericos já são do tipo inteiro, com execção do kilometro

In [25]:
# df = gp.read_file('C:/Users/crash/Desktop/raw_ingestao_mobile.shp')

df["date"] = df["fonte"].str[106:116]
df["quarter"] = df["fonte"].str[104:105]
df = df.drop('fonte', axis=1)
# df = df.drop('geometry', axis=1)
# df = df.drop('Unnamed: 0', axis=1)
# df['avg_d_mbps'] = df['avg_d_kbps'] / 1024
# df['avg_d_gbps'] = df['avg_d_mbps'] / 1024
# df['avg_u_mbps'] = df['avg_u_kbps'] / 1024
# df['avg_u_gbps'] = df['avg_u_mbps'] / 1024
df['network_type'] = 'mobile'
df.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,geometry,latitude,longitude,date,quarter,network_type
0,0323230233223102,15591,17249,26,4,2,138,1400100,"POLYGON ((-60.75439 2.84978, -60.74890 2.84978...",2.847033,-60.751648,2022-01-01,1,mobile
1,0323230233232230,16998,20375,25,2,1,138,1400100,"POLYGON ((-60.72144 2.82234, -60.71594 2.82234...",2.819601,-60.718689,2022-01-01,1,mobile
2,0323232011001023,17268,8038,29,2,2,138,1400100,"POLYGON ((-60.77087 2.79491, -60.76538 2.79491...",2.792168,-60.768127,2022-01-01,1,mobile
3,0323230233232011,116923,4563,36,1,1,138,1400100,"POLYGON ((-60.71594 2.85526, -60.71045 2.85526...",2.852520,-60.713196,2022-01-01,1,mobile
4,0323230233232033,5092,7704,31,2,2,138,1400100,"POLYGON ((-60.71594 2.83880, -60.71045 2.83880...",2.836060,-60.713196,2022-01-01,1,mobile


In [26]:
df.to_file('trusted_speedtest_mobile.shp', driver='ESRI Shapefile')

C:\Users\crash\AppData\Local\Temp\ipykernel_18552\3810273202.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file('trusted_speedtest_mobile.shp', driver='ESRI Shapefile')
